⚠️ NOTE:
This notebook is for research and validation only.
Production logic lives in the `src/` directory.


In [1]:
# Project path setup
import sys
from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\shubh\crypto-market-opportunity-engine")
sys.path.insert(0, str(PROJECT_ROOT))

print("✅ Project root added:", PROJECT_ROOT)


✅ Project root added: C:\Users\shubh\crypto-market-opportunity-engine


In [3]:
import pandas as pd
import numpy as np

from src.inference import load_model
import src.config as config


In [7]:
# load signal data
DATA_PROCESSED = r"C:\Users\shubh\crypto-market-opportunity-engine\data\processed\BTCUSDT_5m_signals.parquet"

df = pd.read_parquet(DATA_PROCESSED)

print("Shape:", df.shape)
print(df[["prob_up", "signal"]].head())


Shape: (105092, 28)
                      prob_up signal
open_time                           
2025-01-01 02:20:00  0.503679   HOLD
2025-01-01 02:25:00  0.504466   HOLD
2025-01-01 02:30:00  0.503574   HOLD
2025-01-01 02:35:00  0.499005   HOLD
2025-01-01 02:40:00  0.492561   HOLD


In [8]:
# backtest assumption
INITIAL_CAPITAL = 100_000  # ₹ or $
FEE_RATE = 0.0004          # 0.04% per trade (Binance-like)


In [9]:
# trade simulation logic
df = df.copy()

df["position"] = 0
df["entry_price"] = np.nan
df["exit_price"] = np.nan
df["pnl"] = 0.0

position = 0
entry_price = 0

for i in range(len(df)):
    signal = df.iloc[i]["signal"]
    price = df.iloc[i]["close"]

    # ENTER
    if signal == "BUY" and position == 0:
        position = 1
        entry_price = price
        df.iloc[i, df.columns.get_loc("position")] = 1
        df.iloc[i, df.columns.get_loc("entry_price")] = price

    # EXIT
    elif signal == "SELL" and position == 1:
        pnl = (price - entry_price) / entry_price
        pnl -= 2 * FEE_RATE  # entry + exit fee

        df.iloc[i, df.columns.get_loc("exit_price")] = price
        df.iloc[i, df.columns.get_loc("pnl")] = pnl

        position = 0
        entry_price = 0


In [10]:
# equity curve
df["strategy_return"] = df["pnl"].fillna(0)
df["equity_curve"] = (1 + df["strategy_return"]).cumprod()

df[["strategy_return", "equity_curve"]].tail()


,strategy_return,equity_curve
open_time,,
2025-12-31 23:35:00,0.0,1.459528
2025-12-31 23:40:00,0.0,1.459528
2025-12-31 23:45:00,0.0,1.459528
2025-12-31 23:50:00,0.0,1.459528
2025-12-31 23:55:00,0.0,1.459528


In [11]:
# performance metrics
trades = df[df["pnl"] != 0]

total_return = df["equity_curve"].iloc[-1] - 1
win_rate = (trades["pnl"] > 0).mean()

max_drawdown = (
    df["equity_curve"] / df["equity_curve"].cummax() - 1
).min()

sharpe = (
    df["strategy_return"].mean()
    / df["strategy_return"].std()
) * np.sqrt(365 * 24 * 12)  # 5-min bars

print(f"📈 Total Return: {total_return:.2%}")
print(f"🎯 Win Rate: {win_rate:.2%}")
print(f"📉 Max Drawdown: {max_drawdown:.2%}")
print(f"⚖️ Sharpe Ratio: {sharpe:.2f}")
print(f"🔁 Trades: {len(trades)}")


📈 Total Return: 45.95%
🎯 Win Rate: 66.67%
📉 Max Drawdown: -9.91%
⚖️ Sharpe Ratio: 1.10
🔁 Trades: 6


In [12]:
# sanity checks
trades[["entry_price", "exit_price", "pnl"]].head(10)


,entry_price,exit_price,pnl
open_time,,,
2025-01-14 08:15:00,NaN,95582.00,-0.001406
2025-03-02 15:35:00,NaN,88119.98,-0.097839
2025-04-02 13:45:00,NaN,86101.20,0.010759
2025-05-13 19:45:00,NaN,104934.23,0.377809
2025-09-18 03:05:00,NaN,117336.65,0.076335
2025-10-05 02:30:00,NaN,124131.59,0.080825
